In [38]:
import os
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.agents import UserProxyAgent
from autogen_agentchat.ui import Console
from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage
from typing import Sequence, List
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("OPEN_ROUTER_API_KEY")

if not api_key:
    print("API KEY is missing.")

# model_client = OpenAIChatCompletionClient(
#     base_url="https://openrouter.ai/api/v1",
#     model="mistralai/mistral-7b-instruct",
#     api_key=api_key,
#     model_info={
#         "family": 'gemini',
#         "vision": True,
#         "function_calling": True,
#         "json_output": False,
#         "structured_output": False
#     }
# )

model_client = OpenAIChatCompletionClient(
    base_url="https://openrouter.ai/api/v1",
    model="openai/gpt-4o",  # ✅ GPT-4o via OpenRouter
    api_key=api_key,
    model_info={
        "family": "openai",             # ✅ OpenAI model family
        "vision": True,                 # ✅ GPT-4o supports vision
        "function_calling": True,       # ✅ Supports function calling
        "json_output": True,            # ✅ GPT-4o outputs valid JSON when prompted
        "structured_output": True       # ✅ For Pydantic or tool calling
    }
)


In [ ]:

# Note: This example uses mock tools instead of real APIs for demonstration purposes
def search_web_tool(query: str) -> str:
    if "2006-2007" in query:
        return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        """
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."

def percentage_change_tool(start: float, end: float) -> float:
    return ((end - start) / start) * 100

planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        WebSearchAgent: Searches for information
        DataAnalystAgent: Performs calculations

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

web_search_agent = AssistantAgent(
    "WebSearchAgent",
    description="An agent for searching information on the web.",
    tools=[search_web_tool],
    model_client=model_client,
    system_message="""
    You are a web search agent.
    Your only tool is search_tool - use it to find information.
    You make only one search call at a time.
    Once you have the results, you never do calculations based on them.
    """,
)

data_analyst_agent = AssistantAgent(
    "DataAnalystAgent",
    description="An agent for performing calculations.",
    model_client=model_client,
    tools=[percentage_change_tool],
    system_message="""
    You are a data analyst.
    Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided.
    If you have not seen the data, ask for it.
    """,
)

text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination

selector_prompt = """Select an agent to perform task. 

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure the planner agent has assigned tasks before other agents start working.
Only select one agent.
"""


def selector_function(messages: Sequence[BaseAgentEvent | BaseChatMessage])-> str:
    if messages[-1].source != web_search_agent.name:
        return planning_agent.name
    
    if messages[-1].source != data_analyst_agent.name:
        return planning_agent.name
    
    return None



team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    selector_func=selector_function,
    allow_repeated_speaker=False
)

task = "Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

# Use asyncio.run(...) if you are running this in a script.
await Console(team.run_stream(task=task))


---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?


---------- TextMessage (PlanningAgent) ----------
To find the information and perform the calculations required, the tasks need to be broken down as follows:

1. WebSearchAgent: Search for the Miami Heat player with the highest points in the 2006-2007 NBA season.
2. WebSearchAgent: Find the total rebounds for this player in the 2007-2008 season.
3. WebSearchAgent: Find the total rebounds for this player in the 2008-2009 season.
4. DataAnalystAgent: Calculate the percentage change in the player's total rebounds between the 2007-2008 and 2008-2009 seasons.
---------- TextMessage (PlanningAgent) ----------
1. WebSearchAgent: Search for the Miami Heat player with the highest points in the 2006-2007 NBA season.
---------- TextMessage (PlanningAgent) ----------
2. WebSearchAgent: Find the total rebounds for the identified Miami Heat player in the 2007-2008 season.
---------- TextMessage (PlanningAgent) ----------
3. WebSearchAgent: Find the total rebounds for the identified Miami Heat player

TaskResult(messages=[TextMessage(id='092f87b6-ae82-4a53-989b-97c744bf47f7', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 20, 0, 0, 50, 27367, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='5407df1b-2381-410e-85a7-d9c93a06d8ef', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=160, completion_tokens=126), metadata={}, created_at=datetime.datetime(2025, 7, 20, 0, 0, 55, 647749, tzinfo=datetime.timezone.utc), content="To find the information and perform the calculations required, the tasks need to be broken down as follows:\n\n1. WebSearchAgent: Search for the Miami Heat player with the highest points in the 2006-2007 NBA season.\n2. WebSearchAgent: Find the total rebounds for this player in the 2007-2008 season.\n3. WebSearc

In [37]:
def candidate_func(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> List[str]:
    # keep planning_agent first one to plan out the tasks
    if messages[-1].source == "user":
        return [planning_agent.name]

    # if previous agent is planning_agent and if it explicitely asks for web_search_agent
    # or data_analyst_agent or both (in-case of re-planning or re-assignment of tasks)
    # then return those specific agents
    last_message = messages[-1]
    if last_message.source == planning_agent.name:
        participants = []
        if web_search_agent.name in last_message.to_text():
            participants.append(web_search_agent.name)
        if data_analyst_agent.name in last_message.to_text():
            participants.append(data_analyst_agent.name)
        if participants:
            return participants  # SelectorGroupChat will select from the remaining two agents.

    # we can assume that the task is finished once the web_search_agent
    # and data_analyst_agent have took their turns, thus we send
    # in planning_agent to terminate the chat
    previous_set_of_agents = set(message.source for message in messages)
    if web_search_agent.name in previous_set_of_agents and data_analyst_agent.name in previous_set_of_agents:
        return [planning_agent.name]

    # if no-conditions are met then return all the agents
    return [planning_agent.name, web_search_agent.name, data_analyst_agent.name]


# Reset the previous team and run the chat again with the selector function.
await team.reset()
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=termination,
    candidate_func=candidate_func,
)

await Console(team.run_stream(task=task))


---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
To address your questions, we need to follow these steps:

1. Find out which Miami Heat player had the highest points in the 2006-2007 season.
2. Determine the total rebounds for that player in the 2007-2008 and 2008-2009 seasons.
3. Calculate the percentage change in total rebounds between these two seasons.

Here is how we will execute this:

1. WebSearchAgent: Search for the player with the highest points on the Miami Heat during the 2006-2007 NBA season.
2. WebSearchAgent: Find the total rebounds for this player during the 2007-2008 NBA season.
3. WebSearchAgent: Find the total rebounds for this player during the 2008-2009 NBA season.
4. DataAnalystAgent: Calculate the percentage change in total rebounds of the ide

TaskResult(messages=[TextMessage(id='d04c37ba-b99d-495a-8a29-d067b45a0bbe', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 20, 0, 14, 30, 304752, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='5cf68ba0-8a43-4676-80d4-17e4178f7345', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=160, completion_tokens=196), metadata={}, created_at=datetime.datetime(2025, 7, 20, 0, 14, 35, 21839, tzinfo=datetime.timezone.utc), content='To address your questions, we need to follow these steps:\n\n1. Find out which Miami Heat player had the highest points in the 2006-2007 season.\n2. Determine the total rebounds for that player in the 2007-2008 and 2008-2009 seasons.\n3. Calculate the percentage change in total rebounds between these two seaso

In [ ]:
user_proxy_agent = UserProxyAgent("UserProxyAgent", description="A proxy for the user to approve or disapprove tasks.")

text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination


def selector_func_with_user_proxy(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> str | None:
    if messages[-1].source != planning_agent.name and messages[-1].source != user_proxy_agent.name:
        # Planning agent should be the first to engage when given a new task, or check progress.
        return planning_agent.name
    if messages[-1].source == planning_agent.name:
        if messages[-2].source == user_proxy_agent.name and "APPROVE" in messages[-1].content.upper():  # type: ignore
            # User has approved the plan, proceed to the next agent.
            return None
        # Use the user proxy agent to get the user's approval to proceed.
        return user_proxy_agent.name
    if messages[-1].source == user_proxy_agent.name:
        # If the user does not approve, return to the planning agent.
        if "APPROVE" not in messages[-1].content.upper():  # type: ignore
            return planning_agent.name
    return None

selector_prompt = """Select an agent to perform task. 

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure the planner agent has assigned tasks before other agents start working.
Only select one agent.

Note: If you are not sure then go back to user aget.

"""
# Reset the previous agents and run the chat again with the user proxy agent and selector function.
await team.reset()
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent, user_proxy_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    selector_func=selector_func_with_user_proxy,
    allow_repeated_speaker=True,
)

await Console(team.run_stream(task=task))


---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
To address this query, we need to:

1. Find out who the Miami Heat player with the highest points in the 2006-2007 season was.
2. Determine his total rebounds for the 2007-2008 and 2008-2009 seasons.
3. Calculate the percentage change in total rebounds between these two seasons.

Tasks:
1. WebSearchAgent: Identify the Miami Heat player with the highest points in the 2006-2007 season.
2. WebSearchAgent: Determine the total rebounds for this player in the 2007-2008 season.
3. WebSearchAgent: Determine the total rebounds for this player in the 2008-2009 season.
4. DataAnalystAgent: Calculate the percentage change in total rebounds from the 2007-2008 season to the 2008-2009 season.
---------- TextMessage (UserProxyAgent) -

TaskResult(messages=[TextMessage(id='5bf5107e-1e0c-44a4-a65f-2a1a3ac21aba', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 20, 0, 23, 35, 944936, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='cc4d9577-7b7e-4370-acbd-07bbad89b1ac', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=160, completion_tokens=176), metadata={}, created_at=datetime.datetime(2025, 7, 20, 0, 23, 39, 688533, tzinfo=datetime.timezone.utc), content='To address this query, we need to:\n\n1. Find out who the Miami Heat player with the highest points in the 2006-2007 season was.\n2. Determine his total rebounds for the 2007-2008 and 2008-2009 seasons.\n3. Calculate the percentage change in total rebounds between these two seasons.\n\nTasks:\n1. WebSearchAge

### Using Reasoning Models

In [ ]:
model_client = OpenAIChatCompletionClient(
    base_url="https://openrouter.ai/api/v1",
    model="openai/o3-mini",  # ✅ o3-mini via OpenRouter
    api_key=api_key,
    model_info={
        "family": "openai",             # ✅ OpenAI model family
        "vision": True,                 # ✅ GPT-4o supports vision
        "function_calling": True,       # ✅ Supports function calling
        "json_output": True,            # ✅ GPT-4o outputs valid JSON when prompted
        "structured_output": True       # ✅ For Pydantic or tool calling
    }
)

web_search_agent = AssistantAgent(
    "WebSearchAgent",
    description="An agent for searching information on the web.",
    tools=[search_web_tool],
    model_client=model_client,
    system_message="""Use web search tool to find information.""",
)

data_analyst_agent = AssistantAgent(
    "DataAnalystAgent",
    description="An agent for performing calculations.",
    model_client=model_client,
    tools=[percentage_change_tool],
    system_message="""Use tool to perform calculation. If you have not seen the data, ask for it.""",
)

user_proxy_agent = UserProxyAgent(
    "UserProxyAgent",
    description="A user to approve or disapprove tasks.",
)

selector_prompt = """Select an agent to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
When the task is complete, let the user approve or disapprove the task.
"""

team = SelectorGroupChat(
    [web_search_agent, data_analyst_agent, user_proxy_agent],
    model_client=model_client,
    termination_condition=termination,  # Use the same termination condition as before.
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,
)

task = "Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

await Console(team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='call_NDhdzNzqCYd1XKVzxuwiWRLH', arguments='{"query": "Miami Heat player with the highest points in the 2006-2007 season and his total rebounds in 2007-2008 and 2008-2009 Dwyane Wade rebounds percentage change"}', name='search_web_tool')]
---------- ToolCallExecutionEvent (WebSearchAgent) ----------
[FunctionExecutionResult(content='Here are the total points scored by Miami Heat players in the 2006-2007 season:\n        Udonis Haslem: 844 points\n        Dwayne Wade: 1397 points\n        James Posey: 550 points\n        ...\n        ', name='search_web_tool', call_id='call_NDhdzNzqCYd1XKVzxuwiWRLH', is_error=False)]
---------- ToolCallSummaryMessage (WebSearchAgent) ----------
Here are the to

e:\Learning\Agentic_AI_V2\autogen-venv\Lib\site-packages\autogen_agentchat\agents\_assistant_agent.py:1118: UserWarning: Finish reason mismatch: stop != tool_calls when tool_calls are present. Finish reason may not be accurate. This may be due to the API used that is not returning the correct finish reason.
  model_result = await model_client.create(


---------- TextMessage (DataAnalystAgent) ----------
I see that Dwyane Wade scored the highest with 1397 points in the 2006-2007 season. Could you please provide his total rebounds for the 2007-2008 and 2008-2009 seasons so we can calculate the percentage change?
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='call_98dBc4FjzuYZylSiu9SPfSh0', arguments='{"query": "Dwyane Wade total rebounds for the 2007-2008 season and 2008-2009 season Miami Heat"}', name='search_web_tool')]
---------- ToolCallExecutionEvent (WebSearchAgent) ----------
[FunctionExecutionResult(content='The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214.', name='search_web_tool', call_id='call_98dBc4FjzuYZylSiu9SPfSh0', is_error=False)]
---------- ToolCallSummaryMessage (WebSearchAgent) ----------
The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214.
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall